In [ ]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [6]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 2


# Check the drug

In [7]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [8]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-11 10:57:50] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-11 10:58:20] ax.service.ax_client: Generated new trial 32 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.660762, 'SL_2': 0.173765, 'SL_3': 0.048006, 'LL_1': 0.150282, 'LL_2': 0.838783, 'LL_3': 0.684656, 'P_1': 1.0, 'P_2': 0.980495, 'P_3': 0.994616, 'S_1': 1.0, 'S_2': 0.593897, 'S_3': 0.261775, 'Water': 0.0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  

11 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,39.500000,354.0,3.91,67.0,0.178437,0.097562,0.066250,0.030187,0.066125,0.151313,0.115562,0.089188,0.070312,0.134875,0.435125,0.258687,0.169875,0.136563
std,4.760952,0.0,0.00,0.0,0.030709,0.074434,0.072943,0.039430,0.052462,0.034846,0.071488,0.082250,0.078081,0.067618,0.313566,0.221032,0.099322,0.191186
min,32.000000,354.0,3.91,67.0,0.125000,0.000000,0.000000,0.004000,0.000000,0.082000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.750000,354.0,3.91,67.0,0.160000,0.029500,0.000000,0.007000,0.023750,0.134000,0.066250,0.000000,0.000000,0.109250,0.206250,0.000000,0.122750,0.000000
50%,39.500000,354.0,3.91,67.0,0.171000,0.102000,0.052500,0.010500,0.050000,0.147500,0.138500,0.132500,0.026000,0.157500,0.454500,0.326500,0.146500,0.000000
75%,43.250000,354.0,3.91,67.0,0.204250,0.143500,0.119000,0.030250,0.113000,0.166000,0.167250,0.162500,0.142500,0.168250,0.651000,0.387500,0.241750,0.282750
max,47.000000,354.0,3.91,67.0,0.233000,0.230000,0.233000,0.117000,0.161000,0.233000,0.205000,0.183000,0.210000,0.230000,0.864000,0.747000,0.354000,0.475000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,39.500000,0.354,0.391,0.067,1.0,0.560351,0.376457,0.160153,0.363214,0.842668,0.684037,0.562500,0.432034,0.780173,0.750000,0.573990,0.345794,0.352668
std,4.760952,0.000,0.000,0.000,0.0,0.412300,0.396880,0.199785,0.266523,0.084788,0.422380,0.512348,0.473953,0.364667,0.447214,0.463353,0.224421,0.474726
min,32.000000,0.354,0.391,0.067,1.0,0.000000,0.000000,0.032009,0.000000,0.658786,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.750000,0.354,0.391,0.067,1.0,0.185011,0.000000,0.046104,0.148742,0.807756,0.313386,0.000000,0.000000,0.748531,0.750000,0.000000,0.163063,0.000000
50%,39.500000,0.354,0.391,0.067,1.0,0.619462,0.316589,0.053493,0.298439,0.830321,0.985982,1.000000,0.157623,0.982544,1.000000,0.748566,0.313948,0.000000
75%,43.250000,0.354,0.391,0.067,1.0,0.999269,0.623628,0.154819,0.621100,0.903117,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.513443,0.943334
max,47.000000,0.354,0.391,0.067,1.0,1.000000,1.000000,0.532563,0.801736,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.717637,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000
mean,39.500000,354.0,3.91,67.0,62.453125,34.146875,23.187500,10.565625,23.143750,52.959375,40.446875,31.215625,24.609375,47.206250,435.125000,258.687500,169.875000,136.56250,0.065625
std,4.760952,0.0,0.00,0.0,10.748263,26.051842,25.530175,13.800371,18.361662,12.196027,25.020788,28.787571,27.328320,23.666445,313.566128,221.032341,99.321616,191.18646,0.388037
min,32.000000,354.0,3.91,67.0,43.750000,0.000000,0.000000,1.400000,0.000000,28.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-0.700000
25%,35.750000,354.0,3.91,67.0,56.000000,10.325000,0.000000,2.450000,8.312500,46.900000,23.187500,0.000000,0.000000,38.237500,206.250000,0.000000,122.750000,0.00000,-0.087500
50%,39.500000,354.0,3.91,67.0,59.850000,35.700000,18.375000,3.675000,17.500000,51.625000,48.475000,46.375000,9.100000,55.125000,454.500000,326.500000,146.500000,0.00000,0.000000
75%,43.250000,354.0,3.91,67.0,71.487500,50.225000,41.650000,10.587500,39.550000,58.100000,58.537500,56.875000,49.875000,58.887500,651.000000,387.500000,241.750000,282.75000,0.350000
max,47.000000,354.0,3.91,67.0,81.550000,80.500000,81.550000,40.950000,56.350000,81.550000,71.750000,64.050000,73.500000,80.500000,864.000000,747.000000,354.000000,475.00000,0.700000


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.0
SL_1       0.5
SL_2       0.4
SL_3       0.2
LL_1       0.4
LL_2       0.8
LL_3       0.6
P_1        0.5
P_2        0.4
P_3        0.8
S_1        7.0
S_2        4.1
S_3        2.7
Water      2.2
Solvent    0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-11 11:09:19] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_2.json`.


# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [16]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [17]:
updated_script

'protocol/iteration_2_OT_2_protocol_.py'

# Perform experiment

# Results

In [18]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,32,354,3.91,67,0.153,0.101,0.027,0.007,0.023,0.128,0.105,0.153,0.150,0.152,0.539,0.320,0.141,0.000,12,0
1,33,354,3.91,67,0.204,0.068,0.117,0.085,0.125,0.183,0.198,0.000,0.007,0.013,0.000,0.381,0.238,0.381,10,0
2,34,354,3.91,67,0.157,0.136,0.001,0.008,0.047,0.147,0.157,0.157,0.033,0.157,0.646,0.000,0.354,0.000,11,0
3,35,354,3.91,67,0.205,0.205,0.013,0.007,0.000,0.162,0.205,0.000,0.000,0.205,0.864,0.080,0.055,0.000,9,0
4,36,354,3.91,67,0.167,0.004,0.165,0.010,0.024,0.136,0.000,0.167,0.167,0.158,0.434,0.434,0.132,0.000,11,0
5,37,354,3.91,67,0.178,0.103,0.137,0.093,0.137,0.143,0.137,0.000,0.019,0.053,0.000,0.382,0.152,0.467,10,0
6,38,354,3.91,67,0.140,0.140,0.000,0.007,0.035,0.117,0.140,0.140,0.140,0.140,0.333,0.333,0.095,0.239,11,0
7,39,354,3.91,67,0.233,0.000,0.233,0.117,0.161,0.233,0.022,0.000,0.000,0.000,0.000,0.369,0.263,0.369,7,0
8,40,354,3.91,67,0.183,0.154,0.000,0.012,0.053,0.148,0.000,0.183,0.107,0.159,0.666,0.000,0.334,0.000,9,0
9,41,354,3.91,67,0.167,0.000,0.000,0.007,0.000,0.159,0.167,0.167,0.167,0.167,0.860,0.000,0.140,0.000,8,0


In [19]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,245.588658,0.252197
1,Good,A2,243.441215,0.288579
2,Good,A3,236.507480,0.333050
3,Caution,A4,352.389641,0.570796
4,Caution,A5,295.659902,0.570796
5,Caution,A6,286.803805,0.570796
6,Good,B1,265.193574,0.312450
7,Good,B2,268.363691,0.275460
8,Good,B3,286.539537,0.348535
9,Caution,B4,271.375064,0.457815


In [20]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,241.845785,4.746156,0.291275,0.040494,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,273.365601,11.518544,0.312148,0.036538,1
3,Formulation 4,272.326015,5.834603,0.349217,0.000344,1
4,Formulation 5,234.517484,3.657149,0.502552,0.037655,1
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,212.966290,2.180908,0.252679,0.029773,1
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,233.168697,9.480227,0.437448,0.035573,1
9,Formulation 10,229.001026,0.299761,0.367915,0.006976,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,32,354,3.91,67,0.153,0.101,0.027,0.007,0.023,0.128,...,0.141,0.000,12,0,Formulation 1,241.845785,4.746156,0.291275,0.040494,1
1,33,354,3.91,67,0.204,0.068,0.117,0.085,0.125,0.183,...,0.238,0.381,10,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,34,354,3.91,67,0.157,0.136,0.001,0.008,0.047,0.147,...,0.354,0.000,11,0,Formulation 3,273.365601,11.518544,0.312148,0.036538,1
3,35,354,3.91,67,0.205,0.205,0.013,0.007,0.000,0.162,...,0.055,0.000,9,0,Formulation 4,272.326015,5.834603,0.349217,0.000344,1
4,36,354,3.91,67,0.167,0.004,0.165,0.010,0.024,0.136,...,0.132,0.000,11,0,Formulation 5,234.517484,3.657149,0.502552,0.037655,1
5,37,354,3.91,67,0.178,0.103,0.137,0.093,0.137,0.143,...,0.152,0.467,10,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,38,354,3.91,67,0.140,0.140,0.000,0.007,0.035,0.117,...,0.095,0.239,11,0,Formulation 7,212.966290,2.180908,0.252679,0.029773,1
7,39,354,3.91,67,0.233,0.000,0.233,0.117,0.161,0.233,...,0.263,0.369,7,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,40,354,3.91,67,0.183,0.154,0.000,0.012,0.053,0.148,...,0.334,0.000,9,0,Formulation 9,233.168697,9.480227,0.437448,0.035573,1
9,41,354,3.91,67,0.167,0.000,0.000,0.007,0.000,0.159,...,0.140,0.000,8,0,Formulation 10,229.001026,0.299761,0.367915,0.006976,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,32,354,3.91,67,0.153,0.101,0.027,0.007,0.023,0.128,...,12,0,Formulation 1,241.845785,4.746156,0.291275,0.040494,1,306.0,0
1,33,354,3.91,67,0.204,0.068,0.117,0.085,0.125,0.183,...,10,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,34,354,3.91,67,0.157,0.136,0.001,0.008,0.047,0.147,...,11,0,Formulation 3,273.365601,11.518544,0.312148,0.036538,1,314.0,0
3,35,354,3.91,67,0.205,0.205,0.013,0.007,0.000,0.162,...,9,0,Formulation 4,272.326015,5.834603,0.349217,0.000344,1,410.0,0
4,36,354,3.91,67,0.167,0.004,0.165,0.010,0.024,0.136,...,11,0,Formulation 5,234.517484,3.657149,0.502552,0.037655,1,334.0,0
5,37,354,3.91,67,0.178,0.103,0.137,0.093,0.137,0.143,...,10,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,38,354,3.91,67,0.140,0.140,0.000,0.007,0.035,0.117,...,11,0,Formulation 7,212.966290,2.180908,0.252679,0.029773,1,280.0,0
7,39,354,3.91,67,0.233,0.000,0.233,0.117,0.161,0.233,...,7,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,40,354,3.91,67,0.183,0.154,0.000,0.012,0.053,0.148,...,9,0,Formulation 9,233.168697,9.480227,0.437448,0.035573,1,366.0,0
9,41,354,3.91,67,0.167,0.000,0.000,0.007,0.000,0.159,...,8,0,Formulation 10,229.001026,0.299761,0.367915,0.006976,1,334.0,0


In [23]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.241846,0.004746,0.153,0.0,0.291275,0.040494,1.000000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.833333
2,1,0.273366,0.011519,0.157,0.0,0.312148,0.036538,0.916667
3,1,0.272326,0.005835,0.205,0.0,0.349217,0.000344,0.750000
4,1,0.234517,0.003657,0.167,0.0,0.502552,0.037655,0.916667
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.833333
6,1,0.212966,0.002181,0.140,0.0,0.252679,0.029773,0.916667
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
8,1,0.233169,0.009480,0.183,0.0,0.437448,0.035573,0.750000
9,1,0.229001,0.000300,0.167,0.0,0.367915,0.006976,0.666667


In [24]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.241846,0.004746,0.153,0.0,0.291275,0.040494,1.000000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.833333
2,1,0.273366,0.011519,0.157,0.0,0.312148,0.036538,0.916667
3,1,0.272326,0.005835,0.205,0.0,0.349217,0.000344,0.750000
4,1,0.234517,0.003657,0.167,0.0,0.502552,0.037655,0.916667
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.833333
6,1,0.212966,0.002181,0.140,0.0,0.252679,0.029773,0.916667
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
8,1,0.233169,0.009480,0.183,0.0,0.437448,0.035573,0.750000
9,1,0.229001,0.000300,0.167,0.0,0.367915,0.006976,0.666667


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,32,354,3.91,67,0.153,0.101,0.027,0.007,0.023,0.128,...,1.000000,0,Formulation 1,0.241846,0.004746,0.291275,0.040494,1,0.153,0
1,33,354,3.91,67,0.204,0.068,0.117,0.085,0.125,0.183,...,0.833333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,34,354,3.91,67,0.157,0.136,0.001,0.008,0.047,0.147,...,0.916667,0,Formulation 3,0.273366,0.011519,0.312148,0.036538,1,0.157,0
3,35,354,3.91,67,0.205,0.205,0.013,0.007,0.000,0.162,...,0.750000,0,Formulation 4,0.272326,0.005835,0.349217,0.000344,1,0.205,0
4,36,354,3.91,67,0.167,0.004,0.165,0.010,0.024,0.136,...,0.916667,0,Formulation 5,0.234517,0.003657,0.502552,0.037655,1,0.167,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-11 13:04:54] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-11 13:04:55] ax.service.ax_client: Completed trial 32 with data: {'Solu': (0.153, 0), 'Size': (0.241846, 0.004746), 'PDI': (0.291275, 0.040494), 'Complexity': (1.0, 0)}.
[INFO 02-11 13:04:55] ax.service.ax_client: Completed trial 33 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.833333, 0)}.
[INFO 02-11 13:04:55] ax.service.ax_client: Completed trial 34 with data: {'Solu': (0.157, 0), 'Size': (0.273366, 0.011519), 'PDI': (0.312148, 0.036538), 'Complexity': (0.916667, 0)}.
[INFO 02-11 13:04:55] ax.service.ax_client: Completed trial 35 with data: {'Solu': (0.205, 0), 'Size': (0.272326, 0.005835), 'PDI': (0.349217, 0.000344), 'Complexity': (0.75, 0)}.
[INFO 02-11 13:04:55] ax.service.ax_client: Completed trial 36 with data: {'Solu': (0.167, 0), 'Size': (0.234517, 0.003657), 'PDI': (0.502552, 0.037655), 'Complexity': (0.916667, 0)}.
[INFO 02-11 13:04:55] ax.service.ax_client: Completed trial 37 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0)